
# 🎤 Full Voice Conversion Colab (RVC WebUI + Demucs + Auto‑Mix)

Этот ноутбук «под ключ»:
1) Подключает Google Drive и создаёт папки проекта.  
2) (Опционально) Разделяет стемы (Demucs).  
3) Ставит **RVC WebUI** и запускает веб‑интерфейс (Gradio) в Colab.  
4) Автосводит `instrumental.wav` + `converted.wav` в финальную песню (FFmpeg).  

> **Как пользоваться**: запускайте ячейки сверху вниз. Для обучения/конвертации откроется веб‑ссылка Gradio (RVC WebUI), где вы загрузите свои записи и сделаете замену вокала.


## 1) Подключаем Google Drive

In [ ]:

from google.colab import drive
drive.mount('/content/drive')



## 2) Создаём рабочие папки на Диске
Структура:
```
MyDrive/RVC_Project/
  train_audio/
  input_song/
  outputs/
  stems/
  models/
```


In [ ]:

import os

BASE = "/content/drive/MyDrive/RVC_Project"
folders = [f"{BASE}/train_audio", f"{BASE}/input_song", f"{BASE}/outputs", f"{BASE}/stems", f"{BASE}/models"]
for d in folders:
    os.makedirs(d, exist_ok=True)
print("Готово. База проекта:", BASE)



## 3) (Опционально) Загрузка файлов напрямую (с телефона/ПК)


In [ ]:

from google.colab import files
import shutil, os
print("Выберите файлы. Они будут перемещены в /RVC_Project/input_song")
uploaded = files.upload()
DEST = "/content/drive/MyDrive/RVC_Project/input_song"
os.makedirs(DEST, exist_ok=True)
for name in uploaded.keys():
    shutil.move(name, f"{DEST}/{name}")
print("Готово →", DEST)



## 4) (Опционально) Разделение стемов (Demucs)


In [ ]:

!pip -q install demucs
import os, glob, shutil, pathlib
INPUT = "/content/drive/MyDrive/RVC_Project/input_song"
STEMS = "/content/drive/MyDrive/RVC_Project/stems"
os.makedirs(STEMS, exist_ok=True)
files = sorted(glob.glob(os.path.join(INPUT, "*")))
assert files, "Положите файл песни в RVC_Project/input_song"
song = files[0]
print("Разделяем:", song)
!python -m demucs.separate -n mdx_extra_q --two-stems=vocals -o "/content" "$song"
out_dir = list(pathlib.Path("/content/separated/mdx_extra_q").glob("*"))[0]
for p in out_dir.glob("*"):
    shutil.move(str(p), os.path.join(STEMS, p.name))
print("Готово. Проверьте:", STEMS)



## 5) Установка и запуск RVC WebUI (Gradio)


In [ ]:

!apt -qq update
!apt -qq install -y ffmpeg
!pip -q install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
%cd /content
!git clone -q https://github.com/RVC-Project/Retrieval-based-Voice-Conversion-WebUI rvc
%cd /content/rvc
!pip -q install -r requirements.txt
print("Готово к запуску WebUI")



### 5.1) Запуск WebUI
После запуска появится ссылка **Gradio**.


In [ ]:

%cd /content/rvc
!python infer-web.py --colab



## 6) Автосведение `instrumental.wav` + `converted.wav`


In [ ]:

PROJECT_DIR = "/content/drive/MyDrive/RVC_Project"
INPUT_DIR   = f"{PROJECT_DIR}/input_song"
OUTPUT_DIR  = f"{PROJECT_DIR}/outputs"
INSTRUMENTAL = f"{INPUT_DIR}/instrumental.wav"
VOCAL        = f"{OUTPUT_DIR}/converted.wav"
FINAL_OUT    = f"{OUTPUT_DIR}/FINAL_mix.wav"
FINAL_MP3    = f"{OUTPUT_DIR}/FINAL_mix.mp3"
OFFSET_SEC  = 0.00
VOC_GAIN_DB = -1.0
INS_GAIN_DB = 0.0
!apt -qq install -y ffmpeg
import subprocess, math
offset_ms = max(0, int(round(OFFSET_SEC * 1000)))
if OFFSET_SEC >= 0:
    delay_filter = f"[1:a]volume={VOC_GAIN_DB}dB,adelay={offset_ms}|{offset_ms}[v]"
    trim_instr   = f"[0:a]volume={INS_GAIN_DB}dB[i]"
else:
    trim_sec     = abs(OFFSET_SEC)
    delay_filter = f"[1:a]atrim=start={trim_sec},asetpts=PTS-STARTPTS,volume={VOC_GAIN_DB}dB[v]"
    trim_instr   = f"[0:a]volume={INS_GAIN_DB}dB[i]"
filter_complex = f"{trim_instr};{delay_filter};[i][v]amix=inputs=2:duration=first:normalize=0:dropout_transition=0,loudnorm=I=-14:TP=-1.5:LRA=11"
cmd_wav = ["ffmpeg","-y","-i",INSTRUMENTAL,"-i",VOCAL,"-filter_complex",filter_complex,"-ar","44100","-ac","2","-c:a","pcm_s16le",FINAL_OUT]
print(">>> Сводим в WAV…")
subprocess.run(cmd_wav, check=True)
print("Готово:", FINAL_OUT)
cmd_mp3 = ["ffmpeg","-y","-i",FINAL_OUT,"-c:a","libmp3lame","-b:a","320k",FINAL_MP3]
subprocess.run(cmd_mp3, check=True)
print("Также сохранено MP3:", FINAL_MP3)



## 7) (Опционально) Sidechain‑компрессия


In [ ]:

PROJECT_DIR = "/content/drive/MyDrive/RVC_Project"
INPUT_DIR   = f"{PROJECT_DIR}/input_song"
OUTPUT_DIR  = f"{PROJECT_DIR}/outputs"
INSTRUMENTAL = f"{INPUT_DIR}/instrumental.wav"
VOCAL        = f"{OUTPUT_DIR}/converted.wav"
FINAL_SC     = f"{OUTPUT_DIR}/FINAL_mix_sidechain.wav"
THRESHOLD,RATIO,ATTACK_MS,RELEASE_MS = -18,4,10,200
import subprocess
sidechain = (f"[0:a][1:a]sidechaincompress=threshold={THRESHOLD}dB:ratio={RATIO}:attack={ATTACK_MS}:release={RELEASE_MS}[sc];"
             f"[sc]loudnorm=I=-14:TP=-1.5:LRA=11")
cmd = ["ffmpeg","-y","-i",INSTRUMENTAL,"-i",VOCAL,"-filter_complex",sidechain,"-ar","44100","-ac","2","-c:a","pcm_s16le",FINAL_SC]
subprocess.run(cmd, check=True)
print("Сохранено:", FINAL_SC)
